In [1]:
!nvidia-smi

Mon Jan  2 03:54:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.203.03   Driver Version: 450.203.03   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 라이브러리 및 파일 불러오기

In [2]:
# load libraries

import os
import numpy as np
import pandas as pd
from statistics import mean

import torch
import torchvision

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from scipy import stats

from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

PROJECT_PATH = os.getenv('HOME') + '/aiffel/project/AIFFELTHON'
MODEL_PATH = os.path.join(PROJECT_PATH, 'weights/om_weights')
DATA_PATH = os.path.join('data')
TRAIN_PATH = os.path.join(DATA_PATH, 'train')
TEST_PATH = os.path.join(DATA_PATH, 'test')
REJECT_PATH = os.path.join(DATA_PATH, 'reject')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) # connected to GPU if 'cuda' is printed here

cuda


In [ ]:
# checking imgs in a folder

for dirpath, dirnames, filenames in os.walk(TRAIN_PATH):
    for i, filename in enumerate(filenames):
        print(os.path.join(dirpath, filename)) # prints file names
        image = Image.open(os.path.join(dirpath, filename), 'r')
        print(f'size: ({image.width}, {image.height}, {image.getbands()})') # prints img info
        plt.imshow(image)
        plt.show()
        if i==4:
            break # print 4 per folder

# Create Functions

In [5]:
# Normalize imgs, resize to 224x224
# Create pipeline
# PyTorch offers various augmentation techniques in torchvision.transforms.Compose

def create_dataloader(path, batch_size, istrain):
    nearest_mode = torchvision.transforms.InterpolationMode.NEAREST
    normalize = torchvision.transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
    )
    train_transformer = torchvision.transforms.Compose([
        torchvision.transforms.Resize((224,224), interpolation=nearest_mode),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.RandomVerticalFlip(),
        torchvision.transforms.ColorJitter(),
        torchvision.transforms.ToTensor(),
        normalize
    ])

    test_transformer = torchvision.transforms.Compose([
        torchvision.transforms.Resize((224,224), interpolation=nearest_mode),
        torchvision.transforms.ToTensor(),
        normalize
    ])
    
    if istrain:
        data = torchvision.datasets.ImageFolder(path, transform=train_transformer)
        dataloader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)
        
    else:
        data = torchvision.datasets.ImageFolder(path, transform=test_transformer)
        dataloader = torch.utils.data.DataLoader(data, shuffle=False)

    return dataloader, data

In [6]:
# creating train dataset

BATCH_SIZE = 64 # changed from 64 to 1

train_loader, _train_data = create_dataloader(TRAIN_PATH, BATCH_SIZE, True)
target_class_num = len(os.listdir(os.path.join(TRAIN_PATH)))

print('target_class_num: ', target_class_num)
print('train: ', _train_data.class_to_idx)

target_class_num:  2
train:  {'augmented_07_inner_cupholder': 0, 'resized_data12_inner_rear_seat': 1}


In [7]:
# checking num of imgs in each class

for rootpath, dirpath, filenames in os.walk(TRAIN_PATH):
    print(f'{rootpath} : {len(filenames)}')

data/train : 0
data/train/resized_data12_inner_rear_seat : 1957
data/train/augmented_07_inner_cupholder : 22000


In [8]:
# creating test dataset

BATCH_SIZE = 64 # changed from 64 to 1

test_loader, _test_data = create_dataloader(TEST_PATH, BATCH_SIZE, False)
target_class_num = len(os.listdir(os.path.join(TEST_PATH)))

print('target_class_num: ', target_class_num)
print('test: ', _test_data.class_to_idx)

target_class_num:  2
test:  {'augmented_07_inner_cupholder': 0, 'resized_data12_inner_rear_seat': 1}


In [9]:
# checking num of imgs in each class

for rootpath, dirpath, filenames in os.walk(TEST_PATH):
    print(f'{rootpath} : {len(filenames)}')

data/test : 0
data/test/resized_data12_inner_rear_seat : 1957
data/test/augmented_07_inner_cupholder : 22000


In [10]:
# metrics from sklearn.metrics

def calculate_metrics(trues, preds):
    accuracy = accuracy_score(trues, preds)
    f1 = f1_score(trues, preds, average='macro')
    precision = precision_score(trues, preds, average='macro')
    recall = recall_score(trues, preds, average='macro')
    return accuracy, f1, precision, recall

In [11]:
# train function

def train(dataloader, net, learning_rate, weight_decay_level, device):
    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(
        net.parameters(),
        lr = learning_rate, 
        weight_decay = weight_decay_level
    )

    net.train()

    train_losses = list()
    train_preds = list()
    train_trues = list()

    for idx, (img, label) in enumerate(dataloader):

        img = img.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()

        out = net(img)

        _, pred = torch.max(out, 1)
        loss = criterion(out, label)

        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())
        train_trues.extend(label.view(-1).cpu().numpy().tolist())
        train_preds.extend(pred.view(-1).cpu().detach().numpy().tolist())

    acc, f1, prec, rec = calculate_metrics(train_trues, train_preds)

    print('\n''====== Training Metrics ======')
    print('Loss: ', mean(train_losses))
    print('Acc: ', acc)
    print('F1: ', f1)
    print('Precision: ', prec)
    print('Recall: ', rec)
    print(confusion_matrix(train_trues, train_preds))

    return net, acc, f1, prec, rec

In [12]:
# test function

def test(dataloader, net, device):

    criterion = torch.nn.CrossEntropyLoss()
    
    net.eval()
    test_losses = list()
    test_trues = list()
    test_preds = list()
    
    with torch.no_grad():
        for idx, (img, label) in enumerate(dataloader):

            img = img.to(device)
            label = label.to(device)

            out = net(img)

            _, pred = torch.max(out, 1)
            loss = criterion(out, label)

            test_losses.append(loss.item())
            test_trues.extend(label.view(-1).cpu().numpy().tolist())
            test_preds.extend(pred.view(-1).cpu().detach().numpy().tolist())

    acc, f1, prec, rec = calculate_metrics(test_trues, test_preds)

    print('====== Test Metrics ======')
    print('Test Loss: ', mean(test_losses))
    print('Test Acc: ', acc)
    print('Test F1: ', f1)
    print('Test Precision: ', prec)
    print('Test Recall: ', rec)
    print(confusion_matrix(test_trues, test_preds))

    return net, acc, f1, prec, rec

In [13]:
# code to save best params based on acc

def train_classifier(net, train_loader, test_loader, n_epochs, learning_rate, weight_decay, device):
    best_test_acc = 0
    
    model_save_path = None
    model_save_base = 'weights'
    if not os.path.exists(model_save_base):
        os.makedirs(model_save_base)
    
    print('>> Start Training Model!')
    for epoch in range(n_epochs):
        
        print('> epoch: ', epoch)

        net, _, _, _, _ = train(train_loader, net, learning_rate, weight_decay, device)
        net, test_acc, _, _, _  = test(test_loader, net, device)

        if test_acc > best_test_acc:

            best_test_acc = test_acc
            test_acc_str = '%.5f' % test_acc

            print('[Notification] Best Model Updated!')
            model_save_path = os.path.join(model_save_base, 'om_da_classifier_acc_' + str(test_acc_str) + '.pth') 
            torch.save(net.state_dict(), model_save_path)
                
    return model_save_path

In [14]:
target_class_num

2

In [15]:
# load pre-trained resnet50

net = torchvision.models.resnet50(pretrained=True)
net.fc = torch.nn.Linear(
    net.fc.in_features,
    target_class_num
)

net.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /aiffel/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [16]:
# result

EPOCHS = 20
LEARNING_RATE = 0.005
WEIGHT_DECAY = 0.0005

saved_weight_path = train_classifier(net, train_loader, test_loader, EPOCHS, LEARNING_RATE, WEIGHT_DECAY, device)

>> Start Training Model!
> epoch:  0

====== Training Metrics ======
Loss:  0.06443814971546331
Acc:  0.9795884292691072
F1:  0.9310264985145935
Precision:  0.9371508161057297
Recall:  0.9251085265016026
[[21785   215]
 [  274  1683]]
====== Test Metrics ======
Test Loss:  0.32896467832219173
Test Acc:  0.9183119756229912
Test F1:  0.47870835781273796
Test Precision:  0.4591559878114956
Test Recall:  0.5
[[22000     0]
 [ 1957     0]]
[Notification] Best Model Updated!
> epoch:  1

====== Training Metrics ======
Loss:  0.055648288301813106
Acc:  0.9821346579287891
F1:  0.9393288108025717
Precision:  0.947897629002494
Recall:  0.931150206717146
[[21826   174]
 [  254  1703]]
====== Test Metrics ======
Test Loss:  0.4679808384426859
Test Acc:  0.9183119756229912
Test F1:  0.47870835781273796
Test Precision:  0.4591559878114956
Test Recall:  0.5
[[22000     0]
 [ 1957     0]]
> epoch:  2

====== Training Metrics ======
Loss:  0.03666753076086752
Acc:  0.988145427223776
F1:  0.960418987266


====== Training Metrics ======
Loss:  0.028553465272455166
Acc:  0.9898150853612723
F1:  0.9659937777919259
Precision:  0.9668654702548576
Recall:  0.9651260510057138
[[21882   118]
 [  126  1831]]
====== Test Metrics ======
Test Loss:  0.06852488154580634
Test Acc:  0.9829277455440999
Test F1:  0.9429311897332564
Test Precision:  0.9442814515328484
Test Recall:  0.9415909555442004
[[21802   198]
 [  211  1746]]


In [18]:
# create confidence function
# need softmax and entropy for to check confidence
# get the highest softmax value out of all softmax values and compute entropy based on the mathematical expression

def get_confidence(net, infer_loader, device):    
    container = list()
    
    with torch.no_grad():
        for idx, (img, label) in enumerate(infer_loader):
            img = img.to(device)
            label = label.to(device)
            out = net(img) 
            out_softmax = torch.softmax(out, 1)

            msp = float(out_softmax.detach().cpu().numpy().max()) # max softmax value

            pA = out_softmax.detach().cpu().numpy() / out_softmax.detach().cpu().numpy().sum()
            entropy = -np.sum( pA * np.log2(pA))

            fname, _ = infer_loader.dataset.samples[idx]
            label = int(label.detach().cpu().numpy())

            tmp_container = {
                'fname':fname,
                'label':label,
                'msp':msp,
                'entropy':entropy
            }
            container.append(tmp_container)
        
    return container

# Extract Activation Vector

In [25]:
# need correct activation vector for openmax
# input val in softmax layer is Activation Vector, so retrieve activation from torch.softmax()

train_loader, _train_data = create_dataloader(TRAIN_PATH, 1, False)
target_class_num = len(os.listdir(TRAIN_PATH))

train_preds = list()
train_actvecs = list()
train_outputs_softmax = list()
train_labels = list()

with torch.no_grad():
    for idx, (img, label) in enumerate(train_loader):
        img = img.to(device)
        label = label.to(device)

        out = net(img)
        out_actvec = out.cpu().detach().numpy()[0]
        out_softmax = torch.softmax(out, 1).cpu().detach().numpy()[0]
        out_pred = int(torch.argmax(out).cpu().detach().numpy())
        out_label = int(label.cpu().detach().numpy())

        train_actvecs.append(out_actvec) # component 1: Activation Vector before softmax
        train_preds.append(out_pred) # componenet 2: preds of each data
        train_outputs_softmax.append(out_softmax) # component 3: softmax of each data
        train_labels.append(out_label) # component 4: labels of each data

train_actvecs = np.asarray(train_actvecs)
train_preds = np.asarray(train_preds)
train_outputs_softmax = np.asarray(train_outputs_softmax)
train_labels = np.asarray(train_labels)

In [26]:
# only using correct activations vectors in OpenMax algorithm

train_correct_actvecs = train_actvecs[train_labels==train_preds]
train_correct_labels = train_labels[train_labels==train_preds]
print('Activation vector: ', train_correct_actvecs.shape)
print('Labels: ', train_correct_labels.shape)

Activation vector:  (23548, 2)
Labels:  (23548,)


# Weibull-Distribution

In [27]:
np.unique(train_labels)

array([0, 1])

In [28]:
# parameters for weibull-dist are 3 = shape, loc, scale, class has 4 parameters, so total of 12 nums

class_means = list()
dist_to_means = list()
mr_models = {}

for class_idx in np.unique(train_labels):
    
    print('class_idx: ', class_idx)
    class_act_vec = train_correct_actvecs[train_correct_labels==class_idx]
    print(class_act_vec.shape)
    
    class_mean = class_act_vec.mean(axis=0)
    class_means.append(class_mean)
    
    dist_to_mean = np.square(class_act_vec - class_mean).sum(axis=1) # compute distance of activation vectors
    dist_to_mean_sorted = np.sort(dist_to_mean).astype(np.float64) # sort based on distance
    dist_to_means.append(dist_to_mean_sorted)

    shape, loc, scale = stats.weibull_max.fit(dist_to_mean_sorted[-100:]) # parameters of furthest 100 act vecs
    
    mr_models[str(class_idx)] = {
        'shape':shape,
        'loc':loc,
        'scale':scale
    }
    
class_means = np.asarray(class_means)

class_idx:  0
(21802, 2)
class_idx:  1
(1746, 2)


In [29]:
def compute_openmax(actvec, class_means, mr_models):
    dist_to_mean = np.square(actvec - class_means).sum(axis=1)

    scores = list()
    for class_idx in range(len(class_means)):
        params = mr_models[str(class_idx)]
        score = stats.weibull_max.cdf(
            dist_to_mean[class_idx],
            params['shape'],
            params['loc'],
            params['scale']
        )
        scores.append(score)
    scores = np.asarray(scores)
    
    weight_on_actvec = 1 - scores # weight of each class
    rev_actvec = np.concatenate([
        weight_on_actvec * actvec, # multiplication of known class
        [((1-weight_on_actvec) * actvec).sum()] # computing unknown class
    ])
    
    openmax_prob = np.exp(rev_actvec) / np.exp(rev_actvec).sum()
    return openmax_prob

In [30]:
def inference(actvec, threshold, target_class_num, class_means, mr_models):
    openmax_prob = compute_openmax(actvec, class_means, mr_models)
    openmax_softmax = np.exp(openmax_prob)/sum(np.exp(openmax_prob))

    pred = np.argmax(openmax_softmax)
    if np.max(openmax_softmax) < threshold:
        pred = target_class_num
    return pred

In [31]:
def inference_dataloader(net, data_loader, threshold, target_class_num, class_means, mr_models, is_reject=False):
    result_preds = list()
    result_labels = list()

    with torch.no_grad():
        for idx, (img, label) in enumerate(data_loader):
            img = img.to(device)
            label = label.to(device)

            out = net(img)
            out_actvec = out.cpu().detach().numpy()[0]
            out_softmax = torch.softmax(out, 1).cpu().detach().numpy()[0]
            out_label = int(label.cpu().detach().numpy())

            pred = inference(out_actvec, threshold, target_class_num, class_means, mr_models)

            result_preds.append(pred)
            if is_reject:
                result_labels.append(target_class_num) # 3
            else:
                result_labels.append(out_label) # 0, 1, 2

    return result_preds, result_labels

In [35]:
# finding right threshold
for i in np.arange(0.1,1,0.1):
    test_preds, test_labels = inference_dataloader(net, test_loader, i, target_class_num, class_means, mr_models)
    reject_preds, reject_labels = inference_dataloader(net, reject_loader, i, target_class_num, class_means, mr_models, is_reject=True)
    print('threshold: ', i)
    print('Test Accuracy: ', accuracy_score(test_labels, test_preds))
    print('Reject Accuracy: ', accuracy_score(reject_labels, reject_preds))

threshold:  0.1
Test Accuracy:  0.982426848102851
Reject Accuracy:  0.0018552875695732839
threshold:  0.2
Test Accuracy:  0.982426848102851
Reject Accuracy:  0.0018552875695732839
threshold:  0.30000000000000004
Test Accuracy:  0.982426848102851
Reject Accuracy:  0.0018552875695732839
threshold:  0.4
Test Accuracy:  0.9721584505572484
Reject Accuracy:  0.061224489795918366
threshold:  0.5
Test Accuracy:  0.9426472429770004
Reject Accuracy:  0.15213358070500926
threshold:  0.6
Test Accuracy:  0.0
Reject Accuracy:  1.0
threshold:  0.7000000000000001
Test Accuracy:  0.0
Reject Accuracy:  1.0
threshold:  0.8
Test Accuracy:  0.0
Reject Accuracy:  1.0
threshold:  0.9
Test Accuracy:  0.0
Reject Accuracy:  1.0


In [36]:
# finding right threshold
for i in np.arange(0.3,0.5,0.01):
    test_preds, test_labels = inference_dataloader(net, test_loader, i, target_class_num, class_means, mr_models)
    reject_preds, reject_labels = inference_dataloader(net, reject_loader, i, target_class_num, class_means, mr_models, is_reject=True)
    print('threshold: ', i)
    print('Test Accuracy: ', accuracy_score(test_labels, test_preds))
    print('Reject Accuracy: ', accuracy_score(reject_labels, reject_preds))

threshold:  0.3
Test Accuracy:  0.982426848102851
Reject Accuracy:  0.0018552875695732839
threshold:  0.31
Test Accuracy:  0.982426848102851
Reject Accuracy:  0.0018552875695732839
threshold:  0.32
Test Accuracy:  0.982426848102851
Reject Accuracy:  0.0018552875695732839
threshold:  0.33
Test Accuracy:  0.982426848102851
Reject Accuracy:  0.0018552875695732839
threshold:  0.34
Test Accuracy:  0.9812163459531661
Reject Accuracy:  0.011131725417439703
threshold:  0.35000000000000003
Test Accuracy:  0.9790040489209835
Reject Accuracy:  0.027829313543599257
threshold:  0.36000000000000004
Test Accuracy:  0.976207371540677
Reject Accuracy:  0.03339517625231911
threshold:  0.37000000000000005
Test Accuracy:  0.9753725424719288
Reject Accuracy:  0.04267161410018553
threshold:  0.38000000000000006
Test Accuracy:  0.9740368159619318
Reject Accuracy:  0.05009276437847866
threshold:  0.39000000000000007
Test Accuracy:  0.9726593479984973
Reject Accuracy:  0.055658627087198514
threshold:  0.400000